In [1]:
import pandas as pd
import altair as alt

In [2]:
df = pd.read_csv("../data/VEH0105/regions.csv")

In [3]:
df = df[(df['BodyType']!="Total") & (df['Keepership']!="Total") & (df['Fuel']!="Total")]

In [4]:
df[[col for col in df.columns[6:]]] = df[[col for col in df.columns[6:]]].apply(pd.to_numeric, errors="coerce")

In [31]:
df.head()

,BodyType,Fuel,Keepership,ONS Sort,ONS Code,ONS Geography,2022 Q3,2022 Q2,2022 Q1,2021 Q4,...,2012 Q1,2011 Q4,2011 Q3,2011 Q2,2011 Q1,2010 Q4,2010 Q3,2010 Q2,2010 Q1,2009 Q4
0,Buses and coaches,Diesel,Company,4,E12000001,North East,4.209,4.204,4.227,4.245,...,5.667,5.678,5.571,5.289,5.253,5.341,5.420,5.377,5.376,5.351
1,Buses and coaches,Diesel,Company,20,E12000002,North West,11.295,11.425,11.424,11.489,...,13.236,13.300,13.390,13.600,13.811,13.837,14.202,14.209,14.452,14.490
2,Buses and coaches,Diesel,Company,68,E12000003,Yorkshire and The Humber,7.898,8.027,8.042,8.056,...,9.968,9.949,10.024,10.144,10.101,10.044,10.154,10.003,10.215,10.327
3,Buses and coaches,Diesel,Company,97,E12000004,East Midlands,6.678,6.681,6.647,6.713,...,10.066,10.024,10.113,10.201,10.327,10.436,10.521,10.453,10.364,10.352
4,Buses and coaches,Diesel,Company,142,E12000005,West Midlands,9.186,9.205,9.194,9.206,...,12.428,12.361,12.675,12.529,12.463,12.411,12.682,12.645,12.576,12.691


In [5]:
bt_fuel = df.groupby(["BodyType", "Fuel"])[["2022 Q3"]].sum().reset_index()

In [6]:
bt_fuel['2022 Q3'] = bt_fuel['2022 Q3'] / 1000

In [7]:
order = ['Diesel', 'Petrol', 'Other fuels']
categories = ['Motorcycles', 'Cars', 'Light goods vehicles', 'Heavy goods vehicles']
#colors = ['#f03e3e', '#1098ad', '#37b24d']

In [8]:
chart = alt.Chart(data=bt_fuel, height=300, width=600, title="UK vehicle numbers by category and fuel type").mark_bar().encode(

alt.Y('BodyType:N', sort=categories, title="Category"),
alt.X('2022 Q3:Q', title="Vehicles (millions)"),
alt.Color('Fuel:N', sort=order), #scale=alt.Scale(domain=order, range=colors)),
alt.Order("color_Fuel_sort_index:Q"),
alt.Tooltip(field='2022 Q3', format=",.1f")
)

In [9]:
chart

alt.Chart(...)

In [10]:
df.groupby(['BodyType', 'Keepership'])[['2022 Q3']].sum().reset_index()

,BodyType,Keepership,2022 Q3
0,Buses and coaches,Company,114.767
1,Buses and coaches,Private,29.031
2,Cars,Company,2779.488
3,Cars,Private,29739.392
4,Heavy goods vehicles,Company,427.741
5,Heavy goods vehicles,Private,104.436
6,Light goods vehicles,Company,2187.075
7,Light goods vehicles,Private,2372.133
8,Motorcycles,Company,39.965
9,Motorcycles,Private,1399.925


In [11]:
cars = df[df['BodyType'] == 'Cars']

In [12]:
keep_fuel = cars.groupby(['Keepership', 'Fuel'])[['2022 Q3']].sum().reset_index()
keep_fuel['2022 Q3'] = keep_fuel['2022 Q3'] / 1000

In [13]:
keep_fuel

,Keepership,Fuel,2022 Q3
0,Company,Diesel,0.755738
1,Company,Other fuels,0.805266
2,Company,Petrol,1.218484
3,Private,Diesel,10.832666
4,Private,Other fuels,1.323874
5,Private,Petrol,17.582852


In [28]:
chart2 = alt.Chart(data=keep_fuel, height=150, width=600, title="UK cars by keepership and fuel type").mark_bar().encode(

alt.Y('Fuel:N', title=None, sort="x"),
alt.X('2022 Q3:Q', title="Cars (millions)"),
alt.Color('Fuel:N', sort=['Other fuels', 'Diesel', 'Petrol']), #scale=alt.Scale(domain=order, range=colors)),
alt.Order("color_Fuel_sort_index:Q"),
alt.Row('Keepership'),
alt.Tooltip(field='2022 Q3', format=",.1f")
)

In [29]:
chart2

alt.Chart(...)

In [33]:
alt_cars = cars[cars['Fuel']=='Other fuels']

In [34]:
geo_alt = alt_cars.groupby(['ONS Geography'])[['2022 Q3']].sum().reset_index()

In [35]:
geo_alt

,ONS Geography,2022 Q3
0,East,206.910
1,East Midlands,124.629
2,London,287.317
3,North East,47.558
4,North West,249.304
5,Northern Ireland,38.684
6,Scotland,139.169
7,South East,425.771
8,South West,232.775
9,Wales,59.232
